In [ ]:
### Real images embeddings
import numpy as np
import cv2
import os
import sys
import insightface
from tqdm.auto import tqdm

In [ ]:
real_images = './samples/'
embedding_real_path_root = './embeddings/real/'

In [ ]:
fr_model = insightface.app.FaceAnalysis(
    providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
ctx_id = 0
fr_model.prepare(ctx_id = ctx_id, det_thresh=0.1)

In [ ]:
real_face_not_found = np.zeros(0, dtype=object, order='C')
for file in tqdm(os.listdir(real_images), file=sys.stdout):
    embedding_real_path = os.path.join(
        embedding_real_path_root, os.path.splitext(file)[0] + '.npy')    
    if os.path.exists(embedding_real_path): # if embedding exist, ignore
        continue
    extension = os.path.splitext(file)[1].lower()
    if '.jpg' or '.png' or '.jpeg' in extension:
        face = fr_model.get(cv2.imread(os.path.join(real_images,file)))
        if face:
            embedding = face[0].normed_embedding
            np.save(embedding_real_path, embedding)
        else: # If not face, append to list over faces not found
            real_face_not_found = np.append(real_face_not_found, (real_images + file))

In [ ]:
if len(real_face_not_found) != 0:
      print("Number of faces not detected in real images:",
            len(real_face_not_found), file=sys.stderr)
      print(*real_face_not_found, sep="\n", file=sys.stderr)